In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv("/content/drive/MyDrive/3rd Year 4th Term/NLP/CA4/UpdatedResumeDataSet.csv")


In [5]:
def clean_resume(txt):
    words = word_tokenize(txt)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words and word.isalpha()]
    stemmer = nltk.stem.PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

In [6]:
df['Resume'] = df['Resume'].apply(clean_resume)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'])

In [10]:
import pickle

pickle.dump(le, open('label_encoder.pkl', 'wb'))

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Resume'])
X_sequences = tokenizer.texts_to_sequences(df['Resume'])
X_padded = pad_sequences(X_sequences, maxlen=100)

In [13]:
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(25, activation='softmax'))

In [16]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_padded, df['Category'], test_size=0.2, random_state=42)

In [18]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
10/10 [==============================] - 8s 345ms/step - loss: 3.2004 - accuracy: 0.2016 - val_loss: 3.1779 - val_accuracy: 0.2403
Epoch 2/10
10/10 [==============================] - 4s 438ms/step - loss: 3.0999 - accuracy: 0.2374 - val_loss: 3.0565 - val_accuracy: 0.0779
Epoch 3/10
10/10 [==============================] - 4s 394ms/step - loss: 2.8090 - accuracy: 0.2683 - val_loss: 2.7234 - val_accuracy: 0.2468
Epoch 4/10
10/10 [==============================] - 3s 304ms/step - loss: 2.4480 - accuracy: 0.3707 - val_loss: 2.5183 - val_accuracy: 0.2922
Epoch 5/10
10/10 [==============================] - 3s 307ms/step - loss: 2.2683 - accuracy: 0.4098 - val_loss: 2.6450 - val_accuracy: 0.2468
Epoch 6/10
10/10 [==============================] - 4s 409ms/step - loss: 2.1357 - accuracy: 0.4455 - val_loss: 2.3718 - val_accuracy: 0.2922
Epoch 7/10
10/10 [==============================] - 4s 406ms/step - loss: 1.9444 - accuracy: 0.4667 - val_loss: 2.0787 - val_accuracy: 0.4026
Epoch 

In [ ]:
model.save('lstm_model.h5')